# Dominion simulations

This is a workspace for running and testing my simulations

### Setup

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random
import sys
%matplotlib inline 

In [2]:
#automatically update dominionMarkov module
if 'dominionMarkov' in sys.modules:
    del sys.modules['dominionMarkov'] 
from dominionMarkov import *

### SQL database

The idea behind the database, is that every time I run a simulation, the results are saved.  That way, when I run the simulation a bunch of times, nothing is lost.

#Create database.  Should only be run once
init_sql_database()

In [12]:
#Print out the bottom row of each table in the database, for testing purposes
con = lite.connect('sim.db')
with con:
    cur = con.cursor()
    cur.execute('select * from sim_types where id = (select max(id) from sim_types)')
    #cur.execute('select * from sim_types')
    print(cur.fetchall())
    cur.execute('select * from sim_results where id+10 > (select max(id) from sim_results)')
    results = cur.fetchall()
    for i in results:
        print(i)
con.close()

[(7, 'Markov Village/Smithy Infinite', 'max_cards', 'max_actions', 'fraction_village', 'fraction_smithies', None, 3)]
(1953, 7, 2, 1000.0, 40.0, 0.9125, 0.0125, None, 3.6228420655532423, 7.499984106000615, 0.0, 0)
(1954, 7, 2, 1000.0, 40.0, 0.9125, 0.0375, None, 14.480031629818042, 14.453559904593927, 0.7492572460874571, 0)
(1955, 7, 2, 1000.0, 40.0, 0.9125, 0.0625, None, 1.5757610014864714, 5.829998600332903, 0.9973020404314236, 0)
(1956, 7, 2, 1000.0, 40.0, 0.9125, 0.0875, None, 0.0, 0.0, 0.9999948551757514, 1)
(1957, 7, 2, 1000.0, 40.0, 0.9375, 0.0125, None, 7.071066838607379, 9.999994369413772, 1.1102230246251563e-16, 0)
(1958, 7, 2, 1000.0, 40.0, 0.9375, 0.0375, None, 3.010807090171335, 7.013720625170148, 0.979972359859576, 0)
(1959, 7, 2, 1000.0, 40.0, 0.9375, 0.0625, None, 0.0, 0.0, 0.9999990452327888, 1)
(1960, 7, 2, 1000.0, 40.0, 0.9625, 0.0125, None, 105.0170319795438, 58.74317122737383, 0.08907129564323657, 0)
(1961, 7, 2, 1000.0, 40.0, 0.9625, 0.0375, None, 0.0, 0.0, 0.9999

In [11]:
#code to delete rows, use with caution
con = lite.connect('sim.db')
with con:
    cur = con.cursor()
    cur.execute('update sim_results set deleted = 1 where sim_type_id = 7 and p2 + p3 >= 1')
con.close()

### Monte Carlo simulations

A Monte Carlo simulation simulates many turns, determining the outcome of each by random number generation.  After many turns, statistics are compiled.  This method is the easiest to write, but also slow to execute and less precise.

First I try a Monte Carlo simulation of an infinite lab/copper deck.  It isn't really infinite, I limit it to 1000 cards drawn, after which point it is assumed to be infinite. The output is a tuple with (mean,stdev,reliability).  "reliability" is the probability of drawing infinite cards.  "mean" and "stdev" are statistics compiled from the finite turns.

In [15]:
for i in range(10):
    labsim = monte_lab_inf(i/10)
    print("L = ",i/10,": ",labsim.sim())

L =  0.0 :  (5.0, 0.0, 0.0)
L =  0.1 :  (5.645, 0.9428547077890682, 0.0)
L =  0.2 :  (6.721, 1.9373071516927811, 0.0)
L =  0.3 :  (8.563, 3.8018457359551014, 0.0)
L =  0.4 :  (14.776, 11.509032279040667, 0.0)
L =  0.5 :  (60.06734006734007, 86.93261422465457, 0.10899999999999999)
L =  0.6 :  (14.945736434108527, 12.188004063927611, 0.871)
L =  0.7 :  (8.833333333333334, 5.2094998693625945, 0.994)
L =  0.8 :  (10.0, 0.0, 0.999)
L =  0.9 :  (nan, nan, 1.0)


Next, we simulate a finite lab/copper deck.  Note that reliability is always zero here, since we always draw finite cards.

In [53]:
for i in range(0,20,2):
    labsim = monte_lab_fin(i,deck_size=i+20)
    print(i,"labs, 20 copper:",labsim.sim())

0 labs, 20 copper: (5.0, 0.0, 0.0)
2 labs, 20 copper: (5.499, 0.6511520559746438, 0.0)
4 labs, 20 copper: (6.086, 1.0385586165450647, 0.0)
6 labs, 20 copper: (6.903, 1.6098419177049677, 0.0)
8 labs, 20 copper: (7.736, 2.0655033284892084, 0.0)
10 labs, 20 copper: (8.877, 2.7371282395971126, 0.0)
12 labs, 20 copper: (10.053, 3.3967912800170668, 0.0)
14 labs, 20 copper: (12.319, 4.406953482849573, 0.0)
16 labs, 20 copper: (14.295, 5.266685390262077, 0.0)
18 labs, 20 copper: (15.979, 5.149617364426216, 0.0)


Next I have a simulation of an infinite village/smithy deck.

In [170]:
for i in range(4):
    for j in range(4):
        smithysim = monte_vsm_inf(i/8,j/8)
        print("V = ",i/8,", S = ", j/8, ": ",smithysim.sim())

V =  0.0 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.0 , S =  0.125 :  (5.65, 0.8749285685128811, 0.0)
V =  0.0 , S =  0.25 :  (5.465, 0.9750769200427216, 0.0)
V =  0.0 , S =  0.375 :  (4.833, 1.2398028069011617, 0.0)
V =  0.125 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.125 , S =  0.125 :  (6.105, 1.6043612436106744, 0.0)
V =  0.125 , S =  0.25 :  (6.593, 2.9084963469119223, 0.0)
V =  0.125 , S =  0.375 :  (5.98, 3.5614042174400815, 0.0)
V =  0.25 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.25 , S =  0.125 :  (7.21, 3.6102492988712016, 0.0)
V =  0.25 , S =  0.25 :  (20.642857142857142, 46.10726072142123, 0.006000000000000005)
V =  0.25 , S =  0.375 :  (29.007534983853606, 55.850050542431525, 0.07099999999999995)
V =  0.375 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.375 , S =  0.125 :  (9.223, 6.565917376878878, 0.0)
V =  0.375 , S =  0.25 :  (13.877952755905511, 17.86169708007739, 0.492)
V =  0.375 , S =  0.375 :  (5.245535714285714, 3.7496997116163735, 0.776)


I can also simulate a more complicated deck, such as a [herald](http://wiki.dominionstrategy.com/index.php/File:Herald.jpg)/copper deck.

In [39]:
for i in range(10):
    heraldsim = monte_herald_inf(i/10)
    print("H = ",i/10,": ",heraldsim.sim())

H =  0.0 :  (5.0, 0.0, 0.0)
H =  0.1 :  (5.057, 0.2444401767304119, 0.0)
H =  0.2 :  (5.298, 0.6878924334516273, 0.0)
H =  0.3 :  (5.883, 1.4809831194176368, 0.0)
H =  0.4 :  (7.21, 3.1054629284536635, 0.0)
H =  0.5 :  (10.814, 7.66794653085166, 0.0)
H =  0.6 :  (42.20820512820513, 60.08483519038193, 0.025000000000000022)
H =  0.7 :  (15.614814814814816, 14.5240465666222, 0.865)
H =  0.8 :  (8.142857142857142, 2.948538205792901, 0.993)
H =  0.9 :  (nan, nan, 1.0)


### Markov Chain simulations

The Markov Chain method has a state vector, which each component of that vector indicating the probability that the game is in that particular state.  The state vector is transformed by matrix multiplication.  This method is harder to write, but is faster and more precise, since it does not require random number generation or compiling statistics.

In [260]:
#automatically update dominionMarkov module
if 'dominionMarkov' in sys.modules:
    del sys.modules['dominionMarkov'] 
from dominionMarkov import *

In [135]:
for i in range(10):
    labsim = markov_lab_inf(i/10)
    print("L = ",i/10,": ",labsim.sim())

L =  0.0 :  (5.0, 0.0, 0.0)
L =  0.1 :  (5.6250000000000018, 0.93749999999998868, 0.0)
L =  0.2 :  (6.666666666666667, 1.9245008972987485, 0.0)
L =  0.3 :  (8.75, 4.0504629365049141, -2.2204460492503131e-16)
L =  0.4 :  (14.999999994517884, 12.247448599504688, 1.0739520384106527e-11)
L =  0.5 :  (61.26251048466375, 90.396339694795529, 0.12562280108481549)
L =  0.6 :  (14.999999994517877, 12.24744859950469, 0.86831275720306023)
L =  0.7 :  (8.7500000000000302, 4.0504629365048936, 0.98554173856131377)
L =  0.8 :  (6.6666666666666554, 1.9245008972987707, 0.9990234375)
L =  0.9 :  (5.625000000002446, 0.93749999999286049, 0.99998306491219158)


In [136]:
for i in range(0,20,2):
    labsim = markov_lab_fin(i,deck_size=i+20)
    print(i,"labs, 20 copper:",labsim.sim())

0 labs, 20 copper: (5.0, 0.0, 0.0)
2 labs, 20 copper: (5.4978354978354975, 0.65050402619088699, 0.0)
4 labs, 20 copper: (6.0940146809712017, 1.0740106620045988, 0.0)
6 labs, 20 copper: (6.81735655648699, 1.5382593318885229, 0.0)
8 labs, 20 copper: (7.707788186049056, 2.0809105285633098, 0.0)
10 labs, 20 copper: (8.8216135022731734, 2.7293413115486929, 0.0)
12 labs, 20 copper: (10.239729767338945, 3.5079199246737707, 0.0)
14 labs, 20 copper: (12.080680276847374, 4.4342153900037538, 0.0)
16 labs, 20 copper: (14.283692909344031, 5.2334039868105382, 0.0)
18 labs, 20 copper: (16.02151072091484, 5.2192000774765379, 0.0)


In [220]:
for i in range(4):
    for j in range(4):
        smithysim = markov_vsm_inf(i/8,j/8)
        print("V = ",i/8,", S = ", j/8, ": ",smithysim.sim())

V =  0.0 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.0 , S =  0.125 :  (5.6536140441894531, 0.86326548799271696, 0.0)
V =  0.0 , S =  0.25 :  (5.466064453125, 0.97362470097352949, 0.0)
V =  0.0 , S =  0.375 :  (4.8211860656738281, 1.2002892254189856, 0.0)
V =  0.125 , S =  0.0 :  (5.0, 0.0, 1.1102230246251565e-16)
V =  0.125 , S =  0.125 :  (6.1033275500807127, 1.7272602377671249, -2.2204460492503131e-16)
V =  0.125 , S =  0.25 :  (6.6102875295107282, 2.9752041102361, 0.0)
V =  0.125 , S =  0.375 :  (6.1168365413422281, 3.5099371295479433, 2.2204460492503131e-16)
V =  0.25 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.25 , S =  0.125 :  (7.1463632631821872, 3.4630370273904649, 0.0)
V =  0.25 , S =  0.25 :  (19.249612861176121, 45.233715956419758, 0.0056278130592014586)
V =  0.25 , S =  0.375 :  (29.882254928410589, 64.9287089000603, 0.057486006635842579)
V =  0.375 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.375 , S =  0.125 :  (9.229300204657088, 6.5606277988320985, 1.1102230246251565e-16)
V =  0.375 , S = 

Let's test if the either the card or action cap make a difference.

In [210]:
for i in np.arange(10,80,10):
    smithysim = markov_vsm_inf(.25,.3,max_actions=i)
    (runtime, results) = smithysim.test_sim()
    print("action cap %d : %.2f minutes," % (i,runtime),results)

action cap 10 : 0.00 minutes, (27.09595030916509, 51.530453222255133, 0.00019392010869201393)
action cap 20 : 0.00 minutes, (32.980260796525499, 79.709799223871897, 0.023464591022624215)
action cap 30 : 0.01 minutes, (28.15762657631041, 67.98841582198628, 0.037479824894897873)
action cap 40 : 0.01 minutes, (27.328838100383471, 65.155251230305723, 0.039257620975426732)
action cap 50 : 0.01 minutes, (27.284496179130628, 64.986490201436823, 0.039343436301642387)
action cap 60 : 0.01 minutes, (27.283613011559527, 64.982978938723264, 0.039345064629140558)
action cap 70 : 0.02 minutes, (27.283606225888818, 64.982951287897464, 0.039345076807078661)


In [216]:
for i in np.arange(1000,5000,1000):
    smithysim = markov_vsm_inf(.25,.3,max_cards=i,max_actions=40)
    (runtime, results) = smithysim.test_sim()
    print("card cap %d : %.2f minutes," % (i,runtime),results)

card cap 1000 : 0.01 minutes, (27.328838100383471, 65.155251230305723, 0.039257620975426732)
card cap 2000 : 0.06 minutes, (38.727858995931541, 118.20229966257014, 0.025933950713817322)
card cap 3000 : 0.14 minutes, (50.433199690406354, 175.53906477986024, 0.017945793225463635)
card cap 4000 : 0.39 minutes, (61.69402539523427, 231.36955036547158, 0.012464522116708232)


It seems I don't really need more than 40 max actions, but the card cap seems to make a bigger difference.  This is expected particularly near the phase transition.

### Collecting data

I've tested the simulations and feel satisfied with them.  Now to run the Markov Chain simulations systematically so I can produce results suitable for plotting.

In [226]:
#estimate 4 minutes runtime
for i in np.arange(0,1,.01):
    labsim = markov_lab_inf(i,max_cards=2000)
    labsim.sim()

In [3]:
#estimate 0 minutes run time
for i in range(21):
    labsim = markov_lab_fin(i,deck_size=i+15)
    labsim.sim()

In [14]:
#estimate 0 minutes run time
for i in range(21,41):
    labsim = markov_lab_fin(i,deck_size=i+15)
    labsim.sim()

In [247]:
#estimated 40 minutes run time
density = 40
for i,j in np.ndindex((density,density)):
    if i+j < density:
        smithysim = markov_vsm_inf(i/density,j/density,max_cards=2000,max_actions=40)
        smithysim.sim()

C:\Users\Tristan\My documents\Github\dominionPhysics\dominionMarkov.py:460: RuntimeWarning: invalid value encountered in double_scalars
  stdev = (sqmean - mean**2)**0.5


OperationalError: no such column: nan

Aborted because of an error.  It seems that sometimes when stdev is very small, it will be nan because of a rounding error.  I fixed it.

In [261]:
#estimated 40 minutes run time
density = 40
for i,j in np.ndindex((density,density)):
    if i+j < density and i > 0:
        smithysim = markov_vsm_inf(i/density,j/density,max_cards=2000,max_actions=40)
        smithysim.sim()

In [13]:
#The figures look too pixelated.  Let's get even more data points
#estimated hour run time
density = 100
for i,j in np.ndindex((density,density)):
    if i+j < density:
        smithysim = markov_vsm_inf(i/density,j/density,max_cards=1000,max_actions=40)
        smithysim.sim()